In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1

In [2]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=2765ca24acbff7612c2daedca4705142a282df632e9fc6eb70f549de85dc8a69
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.7 MB/s eta 0:00:00


In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00


In [5]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from langchain import HuggingFacePipeline

model_name = "CleverShovel/falcon-7b-instruct-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, remote_code_true=True)

pipeline = transformers.pipeline(
    "text-generation",
    model = model,
    torch_dtype = torch.float16,
    device_map = "auto",
    do_sample = False,
    top_k = 1,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id,
    max_length=200,
    trust_remote_code=True
)

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

template='''[INST] <>
Only tell me the product names. The answer should only include ten names.
<>
{prompt}[/INST]'''

prompt_template = PromptTemplate(template=template, input_variables=["prompt"])

llm = HuggingFacePipeline(pipeline=pipeline)

llm_chain = LLMChain(prompt=prompt_template, llm=llm)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
!pip install faiss-gpu

In [7]:
!pip install --upgrade sentence_transformers

In [10]:
import pandas as pd
products = pd.read_csv('/content/AjioFasionClothing2.csv', usecols=['Product_URL'])

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cuda'})

product_names = products['Product_URL'].values.astype(str)
product_embeddings = FAISS.from_texts(product_names, embeddings)

In [11]:
product_embeddings.similarity_search_with_score("black blazer with product url",k = 10)

[(Document(page_content='https://www.ajio.com/tahvo-solid-slim-fit-blazer/p/464058048_black', metadata={}),
  0.7644827),
 (Document(page_content='https://www.ajio.com/mufti-blazer-with-notched-lapel/p/460828108_black', metadata={}),
  0.7944856),
 (Document(page_content='https://www.ajio.com/van-heusen-slim-fit-blazer-with-trouser-set/p/460373641_black', metadata={}),
  0.8115159),
 (Document(page_content='https://www.ajio.com/louis-philippe-slim-fit-blazer-with-notched-lapel/p/460640468_black', metadata={}),
  0.83252347),
 (Document(page_content='https://www.ajio.com/tahvo-slim-fit-blazer-with-side-vents/p/464059151_black', metadata={}),
  0.8520209),
 (Document(page_content='https://www.ajio.com/klotthe-full-sleeve-blazer-with-pocket-square/p/464234878_white', metadata={}),
  0.8571897),
 (Document(page_content='https://www.ajio.com/van-heusen-slim-fit-blazer-with-trouser-set/p/460373652_grey', metadata={}),
  0.8729971),
 (Document(page_content='https://www.ajio.com/blackberrys-te